# Visualization Attack results

This code contains functions to obtain different graphics about the cost/performance of the tested attacks.

In [ ]:
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import json

import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pickle

In [ ]:
# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

# List for changing quick paths.
workers = ["Victor","Alex", "Local"]

# Change this number to modify path selection.
worker = workers[0]

if worker == "Victor":
  path= "/content/gdrive/MyDrive/Q7/CBI/ANN"
elif worker == "Alex":
  path = "/content/gdrive/MyDrive/CBI/ANN"
else: path = os.getcwd()

# Reading the results.

In [ ]:
results_path=path+"/results"
file_name= "results.json"

with open(results_path+"/"+file_name, 'rb') as handle:
    # results = pickle.load(handle)
    results = json.load(handle)

global distances 
distances = [80*i for i in range(1,26)]

## Modifiying matrix diagonals to 0 (this modification will be avoided in the next result computation execution)
---
This modification is performed as, when sending attacker's data from the same distance as the true emisor, no modification is needed.

In [ ]:
for k,v in results.items():
  for i in range(len(v)):
    v[i][i] = 0

# Visualizing the results.

In [ ]:
'''
%%capture
!pip install jupyter-dash
!pip install "jupyterlab>=1.0" jupyterlab-dash==0.1.0a3
!jupyter labextension install jupyterlab-dash@0.1.0-alpha.3
'''

In [ ]:
'''
%%capture
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
'''

In [ ]:
# Parameters used in the plot.
alphas = [1,0.25,0.1]
polynomial_degrees = [1,3,5]
methods = ["All", "Cov", "Var"]

In [ ]:
# Maximum MAE of all the configurations.
max_val = 0
for res in results.values():
 max_val = max(max_val,np.max(res))

In [ ]:
# Creation of the interactive plot.

%%writefile my_app1.py
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Attack Results"),
    dcc.Graph(id="graph",
              style={
            'height': 600,
            'width': 600,
            "display": "block",
            "margin-left": "auto",
            "margin-right": "auto",
            }),
    html.Label([
          "% of available data per distance",
          dcc.Slider(
                  id='alpha-input',
                  min=0,
                  max=2,
                  step=None,
                  marks={
                    0: '100%',
                    1: '25%',
                    2: '10%'    
              },
              value=0
          )
        ]),  
    html.Label([
      "Polynomial degree",
      dcc.Slider(
              id='pol-input',
              min=0,
              max=2,
              step=None,
              marks={
                0: '1',
                1: '3',
                2: '5'    
          },
          value=0
      )
    ]),
    html.Label([
      "Attack Method",
      dcc.Slider(
              id='met-input',
              min=0,
              max=2,
              step=None,
              marks={
                0: 'All',
                1: 'Cov Only',
                2: 'Var Only'    
          },
          value=0
      )
    ])  
],style={'textAlign': 'center'})

@app.callback(
    Output("graph", "figure"), 
    [Input("alpha-input", "value"),
     Input("pol-input", "value"),
     Input("met-input","value")])
def filter_heatmap(alpha_idx,pol_idx,met_idx):
    alpha = alphas[alpha_idx]
    pol = polynomial_degrees[pol_idx]
    met = methods[met_idx]
    fig = px.imshow(results[str(alpha)+str(pol)+met],
                   labels=dict(x="Target distance", y="Original distance", color="MAE"),
                   x= distances,
                   y= distances,
                   range_color = (0,max_val),
                   width=600, height=600,
    )
    
    fig.update_xaxes(side="top",type= "category")
    fig.update_yaxes(type= "category")
                                    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
!python my_app1.py

# Adding betas

In [ ]:
file_name= "random_reduced.pickle"
with open(results_path+"/"+file_name, 'rb') as handle:
    results = pickle.load(handle)

In [ ]:
# Parameters used in the plot.
alphas = [1,0.25,0.1]
betas = [18,12,6]
polynomial_degrees = [1,3,5]
methods = ["All", "Cov", "Var"]

In [ ]:
# Maximum MAE of all the configurations.
max_val = 0
for res in results.values():
 max_val = max(max_val,np.max(res))

In [ ]:
alphas = [1,0.25,0.1]
betas = [25,12,6]
polynomial_degrees = [1,3,5]
methods = ["All", "Cov", "Var"]

#---------------------------------

alpha = alphas[1] 
beta = betas[1] 
pol = polynomial_degrees[1]
met = methods[0]

In [ ]:
for k,v in results.items():
  for i in range(25):
    for j in range(25):
      v[i,j] = np.abs(v[i,j] - target_errors[j])


for k,v in results.items():
  for i in range(25):
    for j in range(25-i):
      v[24-i,j] = 0

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=3, x_title=r'$\deg(P(x)) = 1  \quad \quad \quad\quad \quad  \quad \deg(P(x)) = 3 \quad \quad \quad \quad \quad \quad \deg(P(x)) = 5$',
                    y_title=r'$\beta  = 6   \qquad \qquad\qquad \qquad  \qquad \beta = 12 \qquad \qquad\qquad \qquad \qquad  \beta = 25$')

for i in range(3):
  for j in range(3):
      beta = betas[i] 
      pol = polynomial_degrees[j]

      img = results[str(0.5)+str(beta)+str(pol)+"All"]
   
      fig.add_trace(go.Heatmap(z = img, x = distances, y = distances, zmax = 300, zmin = 0 , zauto= False),row=i+1, col=j+1)


fig.update_layout(height=800, width=800)

fig

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=9)

for i in range(3):
  for j in range(3):
    for k in range(3):
      alpha = alphas[0]
      met = methods[i]
      beta = betas[j] 
      pol = polynomial_degrees[k]

      img = results[str(alpha)+str(beta)+str(pol)+met]
   
      fig.add_trace(go.Heatmap(z = img, x = distances, y = distances, zmax = 400),row=i+1, col=j*3 + k +1)
fig

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=9)

for i in range(3):
  for j in range(3):
    for k in range(3):
      alpha = alphas[1]
      met = methods[i]
      beta = betas[j] 
      pol = polynomial_degrees[k]

      img = results[str(alpha)+str(beta)+str(pol)+met]
   
      fig.add_trace(go.Heatmap(z = img, x = distances, y = distances, zmax = 400),row=i+1, col=j*3 + k +1)
fig

In [ ]:
# NNet errors for non-modified data (from 80 to 2k km)
target_errors = [9.943584747314453,
 10.873639526367187,
 14.787281494140625,
 12.084732666015626,
 10.136173095703125,
 15.00669921875,
 15.51007080078125,
 16.08845458984375,
 20.5009716796875,
 15.53055908203125,
 17.16761962890625,
 17.71912109375,
 13.44939208984375,
 16.436572265625,
 19.1457861328125,
 25.7074609375,
 36.4585595703125,
 25.6909619140625,
 23.524921875,
 26.1574609375,
 21.4464599609375,
 21.3863818359375,
 35.7403759765625,
 23.459453125,
 16.943662109374845]

In [ ]:
# Reading the modified data (depending on the used method and hyperparameter combination)

with open(path+"/modifiedData/GD/EM_alpha1.pkl", 'rb') as handle:
    GD1 = pickle.load(handle)

with open(path+"/modifiedData/GD/EM_alpha0.5.pkl", 'rb') as handle:
    GD2 = pickle.load(handle)

with open(path+"/modifiedData/GD/EM_alpha0.25.pkl", 'rb') as handle:
    GD3 = pickle.load(handle)

with open(path+"/modifiedData/GD/EM_alpha0.1pkl", 'rb') as handle:
    GD4 = pickle.load(handle)

with open(path+"/modifiedData/Z/EM_alpha1.pkl", 'rb') as handle:
    Z1 = pickle.load(handle)

with open(path+"/modifiedData/Z/EM_alpha0.5pkl", 'rb') as handle:
    Z2 = pickle.load(handle)

with open(path+"/modifiedData/Z/EM_alpha0.25pkl", 'rb') as handle:
    Z3 = pickle.load(handle)

with open(path+"/modifiedData/Z/EM_alpha0.1.pkl", 'rb') as handle:
    Z4 = pickle.load(handle)


In [ ]:
EMs = []

values = [GD1,GD2,GD3,GD4,Z1,Z2,Z3,Z4]

for k in range(len(values)):
  EM = [[0 for i in range(25)] for j in range(25)]
  for i in range(24):
      EM[i][i+1:]  = values[k][i]
  EMs.append(EM)

  for i in range(25):
    for j in range(25):
      EMs[k][i][j] = np.abs(EMs[k][i][j] - target_errors[j])

  for i in range(25):
    for j in range(25-i):
        EMs[k][24-i][j] = 0

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=4, x_title=r'$D^{(d)}_{Att} = 25  \qquad \qquad\qquad\qquad D^{(d)}_{Att} = 12  \qquad \qquad\qquad\qquad  D^{(d)}_{Att} = 6 \qquad \qquad\qquad\qquad  D^{(d)}_{Att} = 2 $', y_title= "Z-score" + " "*46 + "GD")

for i in range(2):
  for j in range(4):

      img = EMs[i*3+j]
   
      fig.add_trace(go.Heatmap(z = img, x = distances, y = distances, zmax = 180, zmin = 0, zauto = False),row=i+1, col=j+1)


fig.update_layout(height=600, width=1000)

fig

# Cost comparison

In [ ]:
for k,v in results.items():
  for i in range(25):
    for j in range(25):
      v[i,j] = np.abs(v[i,j] - target_errors[j])

In [ ]:
for k,v in results.items():
  for i in range(25):
    for j in range(25-i):
      v[24-i,j] = 0

In [ ]:
for i in range(3):
  for j in range(3):
      beta = betas[i] 
      pol = polynomial_degrees[j]
      img = results[str(0.5)+str(beta)+str(pol)+"All"]

In [ ]:
comparar2 = [0 for i in range(9)]
cont = 0 
for i in range(3):
  for j in range(3):
      beta = betas[i] 
      pol = polynomial_degrees[j]
      img = results[str(0.5)+str(beta)+str(pol)+"All"]
      comparar2[cont] = sum(sum(img))*beta/(300*25)
      cont += 1

In [ ]:
plot_comparacion_betas = {"Metric": comparar2,"p":polynomial_degrees*3,"Beta": np.repeat(betas,3)}

plot_comparacion_betas = pd.DataFrame(plot_comparacion_betas)

In [ ]:
fig = px.line(plot_comparacion_betas, x="Beta", y="Metric", color='p', category_orders={"Beta":[6,12,25]})

fig.update_layout(
    xaxis_title=r'$\beta$',
    yaxis_title=r'$\upsilon_{p,\beta}$',
)

fig.update_xaxes(type='category')

fig.update_layout(width=800)

fig.show()